Installing Tweepy **Library**

In [ ]:
!pip install tweepy

Downloading required Packages

In [ ]:
# import tweepy
import tweepy as tw
import numpy as np
import pandas as pd
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

TWITTER API Keys

In [ ]:
#Twitter API key and API secret
api = "vgU87NCZEPxg3lr6bNvf2xlHA"
secretapi = "bUgGuPRupiystGHpntGYfa6F7VcDmzubrzVB1grQuAw34dmEkb"

AUTHENTICATION

In [ ]:
# authenticate
auth = tw.OAuthHandler(api, secretapi)
api = tw.API(auth, wait_on_rate_limit=True)

Getting tweets based on hashtag

In [ ]:
search_query = "#AAPL -filter:retweets"
# getting tweets from the API
tweets = tw.Cursor(api.search_tweets,q=search_query,lang="en").items(500)
# storing the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
print("Total Tweets fetched:", len(tweets_copy))



Forbidden: ignored

Here requesting tweets from twitter requires premium V2 API so I am going extract these tweets by third party software and will use it for training and testing

Financial Data that consists of tweets and Sentiment labels

In [ ]:
import pandas as pd
train_data=pd.read_csv("/content/drive/MyDrive/twitter/stock_data.csv")
print(train_data)

                                                   Text  Sentiment
0     Kickers on my watchlist XIDE TIT SOQ PNK CPW B...          1
1     user: AAP MOVIE. 55% return for the FEA/GEED i...          1
2     user I'd be afraid to short AMZN - they are lo...          1
3                                     MNTA Over 12.00            1
4                                      OI  Over 21.37            1
...                                                 ...        ...
5786  Industry body CII said #discoms are likely to ...         -1
5787  #Gold prices slip below Rs 46,000 as #investor...         -1
5788  Workers at Bajaj Auto have agreed to a 10% wag...          1
5789  #Sharemarket LIVE: Sensex off day’s high, up 6...          1
5790  #Sensex, #Nifty climb off day's highs, still u...          1

[5791 rows x 2 columns]


Downloading required packages

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Preprocessing the tweets

In [ ]:
pre_data=train_data["Text"].tolist()
filtered_statements=[]
stop_words = nltk.corpus.stopwords.words('english')
for statement in pre_data:
    words = statement.split()
    words = [word.lower() for word in words]
    filtered_words = [word for word in words if word not in stop_words]
    filtered_statement = ' '.join(filtered_words)
    filtered_statement = re.sub('','',filtered_statement)
    filtered_statement = re.sub('https://.*','',filtered_statement)
    filtered_statement = re.sub(r'[#$(@]\w+', '', filtered_statement)
    filtered_statements.append(filtered_statement)
print(filtered_statements)

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Lemmatization

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
filterdata=pd.DataFrame(filtered_statements,columns=["tweet"])
filterdata["tweet"]= filterdata.tweet.apply(lemmatize_text)


In [ ]:
filtered = filterdata['tweet'].values
labels = train_data['Sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

[1 1 1 ... 1 1 1]
[1 1 1 ... 1 1 1]


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(filtered,encoded_labels,test_size=0.7)

Tokenizing: Keras’ inbuilt tokenizer API has fit the dataset, which splits the sentences into words and creates a dictionary of all unique words found and their uniquely assigned integers. Each sentence is converted into an array of integers representing all the individual words present in it.
Sequence Padding: The array representing each sentence in the dataset is filled with zeroes to the left to make the size of the array ten and bring all collections to the same length.

In [ ]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
# convert train dataset to sequence and pad sequences
x_train = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(x_train, padding='post', maxlen=max_length)
# convert Test dataset to sequence and pad sequences
x_test = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(x_test, padding='post', maxlen=max_length)
print(test_padded)

[[1304   85    1 ...    0    0    0]
 [   2  246   41 ...    0    0    0]
 [1178   71    1 ...    0    0    0]
 ...
 [ 340  326  427 ...    0    0    0]
 [ 209  162   42 ...    0    0    0]
 [2921   21   86 ...    0    0    0]]


Training the LSTM Model

In [ ]:
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 100)          300000    
                                                                 
 bidirectional (Bidirection  (None, 128)               84480     
 al)                                                             
                                                                 
 dense (Dense)               (None, 24)                3096      
                                                                 
 dense_1 (Dense)             (None, 1)                 25        
                                                                 
Total params: 387601 (1.48 MB)
Trainable params: 387601 (1.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
num_epochs = 10
history = model.fit(train_padded, y_train,
                    epochs=num_epochs, verbose=1,
                    validation_split=0.1)

Epoch 1/10
49/49 [==============================] - 15s 300ms/step - loss: 0.0237 - accuracy: 0.9936 - val_loss: 1.1529 - val_accuracy: 0.7011
Epoch 2/10
49/49 [==============================] - 13s 261ms/step - loss: 0.0104 - accuracy: 0.9987 - val_loss: 1.2313 - val_accuracy: 0.7471
Epoch 3/10
49/49 [==============================] - 14s 279ms/step - loss: 0.0053 - accuracy: 0.9994 - val_loss: 1.3688 - val_accuracy: 0.7299
Epoch 4/10
49/49 [==============================] - 14s 278ms/step - loss: 0.0028 - accuracy: 1.0000 - val_loss: 1.5816 - val_accuracy: 0.7069
Epoch 5/10
49/49 [==============================] - 14s 280ms/step - loss: 0.0015 - accuracy: 1.0000 - val_loss: 1.6919 - val_accuracy: 0.7011
Epoch 6/10
49/49 [==============================] - 14s 278ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 1.7737 - val_accuracy: 0.7126
Epoch 7/10
49/49 [==============================] - 14s 284ms/step - loss: 7.1156e-04 - accuracy: 1.0000 - val_loss: 1.7945 - val_accuracy: 0.

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
prediction = model.predict(test_padded)
# Get labels based on probability 1 if p>= 0.5 else 0
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(y_test,pred_labels))


127/127 [==============================] - 11s 72ms/step
Accuracy of prediction on test set :  0.7276763690182536


In [ ]:
print(confusion_matrix(y_test,pred_labels))

[[ 861  634]
 [ 470 2089]]


Unseen Data

In [ ]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/twitter/unseen_data.csv")
print(data)



      Unnamed: 0                                              Tweet  Sentiment
0           3164  ssys if fundamentals matter... headed under 40...         -1
1           2651  GAE 1.90 now may buy more 1.93 - gut shot to 2.00          1
2           5736  SBI to invest up to 49% equity in Yes Bank, ot...          1
3            217  NVDA the bounce continues and the stock has no...          1
4           1484  been holding this ZNGA long since like dec.. 2...          1
...          ...                                                ...        ...
3470        4604  CA continuing to work well. Glad I bought a de...         -1
3471        5004  Heard on the Street: Investors are betting on ...          1
3472         151                            EOG PXD OAS #energyswag          1
3473        1410  GPN Inverted Head and Shoulders still in play ...          1
3474        4878   we all know what happens to broken momo plays...         -1

[3475 rows x 3 columns]


In [ ]:
tweets=data["Tweet"]
labels=data["Sentiment"].values

0       ssys if fundamentals matter... headed under 40...
1       GAE 1.90 now may buy more 1.93 - gut shot to 2.00
2       SBI to invest up to 49% equity in Yes Bank, ot...
3       NVDA the bounce continues and the stock has no...
4       been holding this ZNGA long since like dec.. 2...
                              ...                        
3470    CA continuing to work well. Glad I bought a de...
3471    Heard on the Street: Investors are betting on ...
3472                              EOG PXD OAS #energyswag
3473    GPN Inverted Head and Shoulders still in play ...
3474     we all know what happens to broken momo plays...
Name: Tweet, Length: 3475, dtype: object
[-1  1  1 ...  1  1 -1]


In [ ]:
filtered_tweets=[]
stop_words = nltk.corpus.stopwords.words('english')
for statement in tweets:
    words = statement.split()
    words = [word.lower() for word in words]
    filtered_words = [word for word in words if word not in stop_words]
    filtered_statement = ' '.join(filtered_words)
    filtered_statement = re.sub('','',filtered_statement)
    filtered_statement = re.sub('https://.*','',filtered_statement)
    filtered_statement = re.sub(r'[#$(@*]\w+', '', filtered_statement)
    filtered_tweets.append(filtered_statement)
print(filtered_tweets)

['ssys fundamentals matter... headed 40. income 10% sales. 40mil net inc. puts pe 60 stock 60. doubt', 'gae 1.90 may buy 1.93 - gut shot 2.00', 'sbi invest 49% equity yes bank, investors also invited: nirmala sitharaman union cabinet meeting', 'nvda bounce continues stock cleared 200sma 13 level. ong', 'holding znga long since like dec.. 2.75-2.70 big resistance.. holds abv itll go 3', 'rbi surpassed market expectations: experts surprise 0.75% rate cut ', 'saudi arabia delayed pricing crude may delivery, cease-fire price war contributed aâ€¦ ', 'spw seeing bearish divergence forming. watch flush short.', 'aap gonna bounce hard 516', "aap > told yesterday buy yet. say buy, i'll backing truck myself.", 'nfx said before...it feel like another distribution day.....', 'podd breaking new 52wk highs decent base', 'mcp look volume v price, transactions around 7 morning = line sand', 'covering 1/2 wac short 47.15, looking lower got book some...', 'green weekly triangle essx,....updating', 'af -

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
filterdata=pd.DataFrame(filtered_tweets,columns=["tweet"])
filterdata["tweet"]= filterdata.tweet.apply(lemmatize_text)
filterdata

,tweet
0,ssys fundamental matter... headed 40. income 1...
1,gae 1.90 may buy 1.93 - gut shot 2.00
2,"sbi invest 49% equity yes bank, investor also ..."
3,nvda bounce continues stock cleared 200sma 13 ...
4,holding znga long since like dec.. 2.75-2.70 b...
...,...
3470,ca continuing work well. glad bought decent si...
3471,heard street: investor betting iacâ€™s remodel...
3472,eog pxd oas
3473,gpn inverted head shoulder still play !!


In [ ]:
x_test = tokenizer.texts_to_sequences(filterdata["tweet"].tolist())
print(x_test)
test_padded = pad_sequences(x_test, padding='post', maxlen=max_length)
print(test_padded)

[[218, 811, 2574, 1238, 153, 1, 27, 1954, 1, 350, 1147, 91, 2146, 207, 6, 207, 1092], [456, 8, 156, 121, 21, 8, 1748, 1, 336, 13, 65], [1, 1, 525, 1957, 1888, 159, 185, 192, 1, 2954, 2955, 2126, 1, 2085], [294, 181, 202, 6, 1312, 947, 149, 109, 28], [101, 137, 11, 90, 9, 1742, 13, 172, 13, 194, 72, 77, 111, 1, 1, 42, 17], [293, 1, 16, 1869, 1670, 1851, 132, 172, 490, 208], [687, 1511, 1, 1, 1365, 121, 1, 1, 933, 39, 1071, 1, 1616], [322, 2299, 164, 600, 573, 25, 1, 4], [2, 908, 181, 246, 1], [2, 1913, 131, 21, 546, 125, 21, 533, 1, 1, 1], [84, 308, 2863, 210, 1650, 9, 136, 1651, 5], [1, 214, 19, 1029, 43, 789, 281], [454, 29, 10, 130, 39, 1, 186, 57, 357, 75, 2777], [1567, 8, 13, 1, 4, 416, 82, 56, 120, 247, 882, 1], [83, 49, 54, 1, 1], [874, 142, 1, 44, 68, 1, 112, 2116, 1757, 49], [72, 105, 249, 1101, 2815, 1, 36, 30, 1], [374, 24, 223], [16, 1794, 321, 1795, 25, 897, 841, 1850, 1057, 303, 204, 1, 1833, 737, 910], [3, 2, 2160, 271, 360, 2161, 2162, 654, 82, 55, 63, 767], [1, 50, 310,

In [ ]:
prediction = model.predict(test_padded)
pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(-1)
print("Accuracy of prediction on test set : ", accuracy_score(labels,pred_labels))
Labels={"actual":labels,"predicted":pred_labels}
print("confusion matrix is \n",confusion_matrix(labels,pred_labels))


109/109 [==============================] - 9s 78ms/step
Accuracy of prediction on test set :  0.7936690647482014
confusion matrix is 
 [[ 858  424]
 [ 293 1900]]
